In [ ]:
import socket
import pickle
from boltons import socketutils
from PIL import ImageGrab
import os
import wmi

server = "localhost"
port = 1234


# dump func
def recvDump(buff):
    dump_size = int(buff.recv_until(b'!!').decode())
    dump = buff.recv_size(dump_size)

    print(f'\tReceived dump data, size: {dump_size}')
    return pickle.loads(dump)

# kill process func
def sendKillProcess(buff, pid):
    print('SEND KILL PROCESS SIGNAL')

    buff.send('killprocess!F!'.encode())
    buff.send(str(pid).encode() + b'!!')
    
# process list func
def getProcessList(buff):
    print('REQUEST PROCESS LIST')

    buff.send('processlist!F!'.encode())
    return recvDump(buff)

# screenshoot func
def getScreenShot(buff):
    print('REQUEST SCREENSHOT')

    buff.send('screenshot!F!'.encode())
    return recvDump(buff)

# close connection func
def sendCloseConection(buff):
    print('SEND CLOSE SIGNAL')

    buff.send('close!F!'.encode())
    buff.close()

c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
c.connect((server, port))

buff = socketutils.BufferedSocket(c, None)

In [ ]:
def sendCommand(buff, command):
    print(f'SEND COMMAND: {command}')

    buff.send('command!F!'.encode())
    buff.send(command.encode() + b'!!')
    
    return recvDump(buff)

In [ ]:
print(sendCommand(buff, 'notepad'))

In [ ]:
#test kill process
sendKillProcess(buff, 7704)

In [ ]:
#test process list
print(getProcessList(buff))

In [ ]:
#test screenshot
getScreenShot(buff).show()

In [ ]:
sendCloseConection(buff)

In [ ]:
os.system('notepad')

In [ ]:
os.startfile('notepad')

In [ ]:
import subprocess
cmd = 'powershell "gps | where {$_.MainWindowTitle } | select Description'
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
for line in proc.stdout:
    if line.rstrip():
        # only print lines that are not empty
        # decode() is necessary to get rid of the binary string (b')
        # rstrip() to remove `\r\n`
        print(line.decode().rstrip())